In [32]:
import rasterio
import pyproj
import os
import math

def get_utm_from_wgs(lon, lat):
    """
    Use longitude, latitude of location for get EPSG code.

    Parameters
    ----------
    lon,lat :
        Longitude, latitude of location you want to get EPSG code

    Returns
    -------
    EPSG code of this location
    """
    utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0' + utm_band
    if lat >= 0:
        epsg_code1 = '326' + utm_band
    else:
        epsg_code1 = '327' + utm_band
    return epsg_code1

def get_bound_image(image_path):
    """
    Get image information from path.

    Parameters
    ----------
    image_path : string
        Path to image file - GeoTiff

    Returns
    -------
    long_min: min Longitude image
    lat_min: min Latitude image
    long_max: max Longitude image
    lat_max: max Latitude image
    transform: Affine instance
        Transform of image.
    """
    with rasterio.open(image_path, mode='r+') as src:
        transform = src.transform
        print(src.width, src.height)
        w, h = src.width, src.height
        projstr = src.crs.to_string()
        check_epsg = src.crs.is_epsg_code
        if check_epsg:
            epsg_code = src.crs.to_epsg()
        else:
            epsg_code = None
    X_res = transform[0]
    Y_res = transform[4]
    trans_X_min, trans_Y_min = transform[2], transform[5]
    trans_X_max, trans_Y_max = trans_X_min + X_res * w, trans_Y_min + Y_res * h
    if epsg_code:
        if epsg_code == 4326:
            long_min, lat_min, long_max, lat_max = trans_X_min, trans_Y_min, trans_X_max, trans_Y_max
        else:
            inproj = pyproj.Proj(init='epsg:{}'.format(epsg_code))
            outproj = pyproj.Proj(init='epsg:{}'.format(4326))
            long_min, lat_min = pyproj.transform(inproj, outproj, trans_X_min, trans_Y_min)
            long_max, lat_max = pyproj.transform(inproj, outproj, trans_X_max, trans_Y_max)
    else:
        inproj = pyproj.Proj(projstr)
        outproj = pyproj.Proj(init='epsg:{}'.format(4326))
        long_min, lat_min = pyproj.transform(inproj, outproj, trans_X_min, trans_Y_min)
        long_max, lat_max = pyproj.transform(inproj, outproj, trans_X_max, trans_Y_max)
    return long_min, lat_min, long_max, lat_max, transform, w, h


def gis_data_latlong_to_utm(long_min, lat_min, long_max, lat_max):
    output_epsg = get_utm_from_wgs(long_min, lat_min)
    inproj = pyproj.Proj(init='epsg:{}'.format(4326))
    outproj = pyproj.Proj(init='epsg:{}'.format(output_epsg))
    trans_X_min_out, trans_Y_min_out = pyproj.transform(inproj, outproj, long_min, lat_min)
    trans_X_max_out, trans_Y_max_out = pyproj.transform(inproj, outproj, long_max, lat_max)
    return trans_X_min_out, trans_Y_min_out, trans_X_max_out, trans_Y_max_out, output_epsg

def get_resolution_meter(image_path):
    long_min, lat_min, long_max, lat_max, transform, w, h = get_bound_image(image_path)
    trans_X_min_out, trans_Y_min_out, trans_X_max_out, trans_Y_max_out, output_epsg = gis_data_latlong_to_utm(long_min, lat_min, long_max, lat_max)
    x_meter = abs(trans_X_min_out - trans_X_max_out)/w
    y_meter = abs(trans_Y_min_out - trans_Y_max_out)/h
    return (math.sqrt(x_meter**2 + y_meter**2)/(math.sqrt(2)))



# fp_aoi = r'E:\TMP_XOA\mongkos_2\box.shp'
# fp_img = r'E:\TMP_XOA\mongkos\mongkos part 3_transparent_mosaic_group1.tif'
# out_dir_img_cut = r'E:\TMP_XOA\mongkos_std\img_cut'
# # clip_aoi_image(fp_img, fp_aoi, out_dir_img_cut)
# resolution = get_resolution_meter(fp_img)

    
    
    

    
# image_dir = r'E:\TMP_XOA\mongkos\mongkos part 3_transparent_mosaic_group1.tif'
# label_dir = r'E:\TMP_XOA\mongkos_2\label.shp'
# box_dir = r'E:\TMP_XOA\mongkos_2\box.shp'
# out_dir = r'E:\TMP_XOA\mongkos_std'
# standardized_shape_epsg(label_dir, image_dir, out_dir)

In [43]:
import os
import random
import rasterio
import numpy as np

from tqdm import *
from osgeo import gdal
from gdalconst import GA_ReadOnly
# import get_resolution_meter

class CD_GenerateTrainingDataset:
    def __init__(self, basefile, labelfile, sampleSize, outputFolder, fileprefix):
        self.basefile=basefile
        # self.imagefile=imagefile
        self.labelfile=labelfile
        self.sampleSize=sampleSize
        self.outputFolder=outputFolder
        self.fileprefix=fileprefix
        self.outputFolder_base=None
        self.outputFolder_image=None
        self.outputFolder_label = None

    def generateTrainingDataset(self, nSamples):
        self.outputFolder_base = os.path.join(self.outputFolder,"image")
        self.outputFolder_label = os.path.join(self.outputFolder,"label")
        if not os.path.exists(self.outputFolder):
            os.makedirs(self.outputFolder)
        if not os.path.exists(self.outputFolder_base):
            os.makedirs(self.outputFolder_base)
        if not os.path.exists(self.outputFolder_label):
            os.makedirs(self.outputFolder_label)
        base=gdal.Open(self.basefile, GA_ReadOnly)

        raster = gdal.Open(self.labelfile, GA_ReadOnly)
        geo = raster.GetGeoTransform()
        proj=raster.GetProjectionRef()
        size_X=raster.RasterXSize
        size_Y=raster.RasterYSize

        rband=np.array(raster.GetRasterBand(1).ReadAsArray())

        icount=0
        with tqdm(total=nSamples) as pbar:
            while icount<nSamples:
                px=random.randint(0,size_X-1-self.sampleSize)
                py=random.randint(0,size_Y-1-self.sampleSize)
                rband = raster.GetRasterBand(1).ReadAsArray(px, py, self.sampleSize, self.sampleSize)
                if np.amax(rband)>0 and np.count_nonzero(rband)>0.005*self.sampleSize*self.sampleSize:
                    geo1=list(geo)
                    geo1[0]=geo[0]+geo[1]*px
                    geo1[3]=geo[3]+geo[5]*py
                    basefile_cr=os.path.join(self.outputFolder_base, self.fileprefix+'_{:03d}.tif'.format(icount+1))
                    gdal.Translate(basefile_cr, base,srcWin = [px,py,self.sampleSize,self.sampleSize])
                    labelfile_cr=os.path.join(self.outputFolder_label, self.fileprefix+'_{:03d}.tif'.format(icount+1))
                    gdal.Translate(labelfile_cr, raster,srcWin = [px,py,self.sampleSize,self.sampleSize])
                    icount+=1
                    pbar.update()
        raster=None
        image=None
        base=None

    def writeLabelAsFile(self, data, filename, geo, proj):
        size_Y, size_X=data.shape
        target_ds = gdal.GetDriverByName('GTiff').Create(filename, size_X, size_Y, 1, gdal.GDT_Byte)
        target_ds.SetGeoTransform(geo)
        target_ds.SetProjection(proj)
        band = target_ds.GetRasterBand(1)
        target_ds.GetRasterBand(1).SetNoDataValue(0)				
        band.WriteArray(data)
        band.FlushCache()

        target_ds=None
        
    def writeDataAsFile(self, data, filename, geo, proj):
        nbands, size_Y, size_X=data.shape
        target_ds = gdal.GetDriverByName('GTiff').Create(filename, size_X, size_Y, nbands, gdal.GDT_Byte)
        target_ds.SetGeoTransform(geo)
        target_ds.SetProjection(proj)
        for i in range(0, nbands):
            band = target_ds.GetRasterBand(i+1)
            band.SetNoDataValue(0)	
            band.WriteArray(data[i,:,:])
            band.FlushCache()

        target_ds=None    
            
def create_list_id(path):
    list_id = []
    for file in os.listdir(path):
        if file.endswith(".tif"):
            list_id.append(file[:-4])
    return list_id

# def create_list_id(dir):
#     list_id = []
#     for fp in glob.glob()

def main_gen_data_with_size(base_path,mask_path,outputFolder, sampleSize=None, gen_them=False):
    list_id = create_list_id(base_path)
    print(list_id)
    for image_id in list_id:
        basefile=os.path.join(base_path,image_id+".tif")
        # imagefile=os.path.join(image_path,image_id+".tif")
        labelfile=os.path.join(mask_path,image_id+".tif")
        resolution = get_resolution_meter(basefile)
        print(image_id,resolution)
        if sampleSize:
            size_cut = sampleSize
        else:
            size_cut = round(0.3*256/(resolution*64))*64
        # size_cut=1024
        print(size_cut)
        with rasterio.open(labelfile) as src:
            w,h = src.width,src.height
        numgen = w*h//((size_cut//2)**2)*2
        print(numgen,'z')
        if gen_them:
            numgen = 200
        print(numgen)
        fileprefix = image_id
        gen=CD_GenerateTrainingDataset(basefile, labelfile, size_cut, outputFolder, fileprefix)
        gen.generateTrainingDataset(numgen)
    return gen.outputFolder_base, gen.outputFolder_label

if __name__=='__main__':
    base_path = r"E:\TMP_XOA\mongkos_std\test\img"
    mask_path = r"E:\TMP_XOA\mongkos_std\test\mask"
    outputFolder = r'E:\TMP_XOA\mongkos_std\test\data_128_False'
    sampleSize = 128
    main_gen_data_with_size(base_path, mask_path, outputFolder, sampleSize=sampleSize, gen_them=False)
    # list_id = create_list_id(base_path)
    # for image_id in list_id:
    #     basefile=os.path.join(base_path,image_id+".tif")
    #     # imagefile=os.path.join(image_path,image_id+".tif")
    #     labelfile=os.path.join(mask_path,image_id+".tif")
    #     sampleSize = None
    #     with rasterio.open(labelfile) as src:
    #         w,h = src.width,src.height
    #     # numgen = (w*h//((sampleSize//4)**2))
    #     fileprefix = image_id
        
    #     # gen=CD_GenerateTrainingDataset(basefile, labelfile, sampleSize, outputFolder, fileprefix)
    #     # gen.generateTrainingDataset(numgen)


['mongkos part 3_transparent_mosaic_group1_0']
2798 1856


c:\Users\SkyMap\anaconda3\envs\geoai\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\Users\SkyMap\anaconda3\envs\geoai\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
c:\Users\SkyMap\anaconda3\envs\geoai\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization m

mongkos part 3_transparent_mosaic_group1_0 0.11289999999974196
128
2534 z
2534


100%|██████████| 2534/2534 [01:32<00:00, 27.32it/s]


In [29]:
fp_aoi = r'E:\TMP_XOA\mongkos_std\box_std.shp'
fp_img = r'E:\TMP_XOA\mongkos\mongkos part 3_transparent_mosaic_group1.tif'
fp_out = r'E:\TMP_XOA\mongkos_std\a.tif'

import rasterio
import numpy as np
import geopandas as gpd


def create_mask_by_shape(fp_shp, fp_img, fp_mask_out):
    df_shp = gpd.read_file(fp_shp)
    list_geo = [(x.geometry) for i, x in df_shp.iterrows()]
    
    with rasterio.open(fp_img) as src:
        tr = src.transform
        width, height = src.width, src.height
        meta = src.meta
    
    mask = rasterio.features.geometry_mask(list_geo,
                                           out_shape=(height, width),
                                           transform=tr,
                                           invert=True,
                                           all_touched=True).astype('uint8')
    meta.update({
        'dtype': 'uint8',
        'count': 1
    })
    with rasterio.open(fp_mask_out, 'w', **meta) as dst:
        dst.write(np.array([mask]))
    

fp_img = r"E:\TMP_XOA\mongkos_std\img_cut\mongkos part 3_transparent_mosaic_group1_0.tif"
fp_shp = r"E:\TMP_XOA\mongkos_std\label_std.shp"
fp_mask_out = r"E:\TMP_XOA\mongkos_std\img_mask\mask.tif"
create_mask_by_shape(fp_shp, fp_img, fp_mask_out)

In [ ]:
image_dir = r'E:\TMP_XOA\mongkos\mongkos part 3_transparent_mosaic_group1.tif'
label_dir = r'E:\TMP_XOA\mongkos_2\label.shp'
box_dir = r'E:\TMP_XOA\mongkos_2\box.shp'
out_dir = r'E:\TMP_XOA\mongkos_std'
standardized_shape_epsg(label_dir, image_dir, out_dir)








In [21]:
import os, glob
def get_file_name_follow_type(dir_source, type_file):
    file_names = []
    if '.'in type_file:
        type_file = '*' + type_file
    else:
        type_file = '*.' + type_file
    for fp in glob.glob(os.path.join(dir_source, type_file)):
        file_names.append(os.path.basename(fp).replace(type_file.replace('*',''), ''))
    return file_names

get_file_name_follow_type(r'E:\TMP_XOA\DuBai\Sentinel2base\new_shp', 'shp')

['duibai_new']

In [24]:
a = 'shpaksk.shp'
a.replace('shp','')

'aksk.'

In [22]:
a.replace('shp','')

'dadasdasd'

In [25]:
type_file = '*.shp'
type_file.replace('*','')

'.shp'

In [1]:
import os
for file in os.listdir(r"E:\TMP_XOA\mongkos_std\tmpp\cut_size\shape_label"):
    if file.startswith("mongkos part 3_transparent_mosaic_group1_0_001"):
        print(file)

In [9]:
image_res = 0.02
h = 512
w = 256
(round(max(h, w) / 64 * 3.2 / 3 * image_res / 0.3)) * 64
(round(min(h, w) / 64 * 3.2 / 3 * image_res / 0.3)) * 64

0

In [2]:
import keras
class TrainingCallback(keras.callbacks.Callback):
    def __init__(self, task_id):
        super(TrainingCallback, self).__init__()
        self.task_id = task_id

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        payload = []
        for key in keys:
            payload.append({
                'key': key,
                'value': str(logs[key])
            })
        payload = {
            'task_id': self.task_id,
            'epoch': epoch + 1,
            'payload': payload
        }
        # url = '{}/internal/training/report'.format(HOSTED_ENDPOINT)
        # print(url)
        # print(payload)
        # request = requests.post(url, json=payload)
        # print(request)
    

In [ ]:
callback = [TrainingCallback(self.task_id)] if self.task_id != -1 else None